# Trading Algorítmico con GRU y Backtrader

Este notebook sigue el tutorial `gru_tutorial.md`. Aquí implementaremos el flujo de trabajo completo: desde la preparación de los datos y el entrenamiento del modelo GRU hasta la creación y ejecución de una estrategia de trading en `backtrader`.

## 1. Cargar y Preparar los Datos

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
import matplotlib.pyplot as plt

# Cargar el conjunto de datos de precios de acciones
df = pd.read_csv('../yahoo/AAPL.csv', index_col='datetime', parse_dates=True)

# Seleccionar la columna 'close' para el análisis
data = df[['close']]

# Escalar los datos al rango [0, 1] para un mejor rendimiento del modelo
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Función para crear secuencias de datos para el modelo GRU
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Definir la longitud de la secuencia para el modelo
SEQ_LENGTH = 60
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Imprimir las formas de los conjuntos de datos para verificar
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

## 2. Construir y Entrenar el Modelo GRU

In [ ]:
# Construir el modelo GRU usando Keras Sequential API
model = Sequential([
    # Primera capa GRU con 50 unidades, retorna secuencias para la siguiente capa
    GRU(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    # Capa de Dropout para regularización y prevenir sobreajuste
    Dropout(0.2),
    # Segunda capa GRU con 50 unidades
    GRU(50, return_sequences=False),
    Dropout(0.2),
    # Capa densa con 25 neuronas
    Dense(25),
    # Capa de salida con una neurona para predecir el precio
    Dense(1)
])

# Compilar el modelo con el optimizador 'adam' y la pérdida de error cuadrático medio
model.compile(optimizer='adam', loss='mean_squared_error')

# Mostrar un resumen de la arquitectura del modelo
model.summary()

# Entrenar el modelo con los datos de entrenamiento y validarlo con los datos de prueba
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

## 3. Guardar el Modelo Entrenado

In [ ]:
# Guardar el modelo entrenado en un archivo HDF5 para su uso posterior
model.save('gru_model.h5')
print('Modelo guardado como gru_model.h5')

## 4. Evaluar el Modelo

In [ ]:
# Realizar predicciones sobre el conjunto de prueba
predictions = model.predict(X_test)
# Invertir la escala de las predicciones para obtener los precios reales
predictions = scaler.inverse_transform(predictions)
# Invertir la escala de los datos de prueba para comparación
y_test_inv = scaler.inverse_transform(y_test)

# Graficar los precios reales vs. los precios predichos
plt.figure(figsize=(14, 5))
plt.plot(data.index[train_size + SEQ_LENGTH:], y_test_inv, color='blue', label='Precio Real')
plt.plot(data.index[train_size + SEQ_LENGTH:], predictions, color='red', label='Precio Predicho')
plt.title('Predicción de Precios de AAPL')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.legend()
plt.show()

## 5. Crear la Estrategia en `backtrader`

In [ ]:
# Importar las bibliotecas necesarias para backtesting
import backtrader as bt
from tensorflow.keras.models import load_model

# Definir la estrategia de trading basada en el modelo GRU
class GRUStrategy(bt.Strategy):
    def __init__(self):
        # Cargar el modelo GRU pre-entrenado
        self.model = load_model('gru_model.h5')
        # Mantener el escalador para transformar los datos
        self.scaler = scaler
        self.seq_length = SEQ_LENGTH
        self.data_close = self.datas[0].close

    def next(self):
        # Asegurarse de que hay suficientes datos para crear una secuencia
        if len(self.data_close) > self.seq_length:
            # Preparar los datos más recientes para la predicción
            last_data = np.array(self.data_close.get(size=self.seq_length))
            last_data_scaled = self.scaler.transform(last_data.reshape(-1, 1))
            X_pred = np.array([last_data_scaled])
            
            # Predecir el precio del siguiente día
            prediction_scaled = self.model.predict(X_pred)
            prediction = self.scaler.inverse_transform(prediction_scaled)[0][0]
            
            # Lógica de trading simple: comprar si la predicción es mayor que el precio actual, vender si es menor
            if prediction > self.data_close[0]:
                if not self.position: # Si no hay una posición abierta
                    self.buy() # Comprar
            elif prediction < self.data_close[0]:
                if self.position: # Si hay una posición abierta
                    self.sell() # Vender

# Instanciar el motor de backtesting de backtrader
cerebro = bt.Cerebro()

# Añadir el feed de datos al motor
data_feed = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data_feed)

# Añadir la estrategia de trading al motor
cerebro.addstrategy(GRUStrategy)

# Configurar el capital inicial y la comisión por operación
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)

# Imprimir el capital inicial y ejecutar el backtest
print(f'Capital Inicial: {cerebro.broker.getvalue()}')
cerebro.run()
# Imprimir el capital final después del backtesting
print(f'Capital Final: {cerebro.broker.getvalue()}')